In [2]:
from os import read
with open("input.txt","r",encoding="utf-8") as f:
  text=f.read()


In [3]:
print("length of dataset in charachters: ",len(text))

length of dataset in charachters:  62037


In [12]:
device="cuda" if torch.cuda.is_available() else "cpu"
print(device)

cuda


In [4]:
print(text[:1000])

Rubai 1:
Ey özünün sırlarına akıl ermeyen;
Suçumuza, duamıza önem vermeyen;
Günahtan sarhoştum, ama dilekten ayık;
Umudumu rahmetine bağlamışım ben

Rubai 2:
Büyükse de isyanım, kötülüklerim,
Yüce Tanrı' dan umut kesmiş değilim;
Bugün sarhoş ve harap ölsem de yarın
Rahmete kavuşur elbet kemiklerim.

Rubai 3:
Tanrım bir geçim kapısı açıver bana;
Kimseye minnetsiz yaşamak yeter bana;
Şarap içir, öyle kendimden geçir ki beni
Haberim olmasın gelen dertten başıma.

Rubai 4:
Rahmetin var, günah işlemekten korkmam;
Azığım senden, yolda çaresiz kalmam;
Mahşerde lutfunla ak pak olursa yüzüm
Defterim kara yazılmış olsun, aldırmam.

Rubai 5:
Derde gama yatkın yüreğime acı;
Bu tutsak cana, garip gönlüme acı;
Bağışla meyhaneye giden ayağımı,
Kızıl kadehi tutan elime acı.

Rubai 6:
Akıl bu kadehi övdükçe över;
Alnından sevgiyle öptükçe öper;
Zaman Usta' ysa bu canım nesneyi
Hem yapar hem kırıp bin parça eder.

Rubai 7:
Ey zaman, bilmez misin ettiğin kötülükleri?
Sana düşer azapların, tövbelerin bete

In [5]:
chars = sorted(list(set(text)))
vocab_size = len(chars)
print("".join(chars))
print(vocab_size)


 !',-.0123456789:;?ABCDEFGHIKLMNOPRSTUVYZabcdefghijklmnoprstuvyzÇÖÜçöüğİıŞş
76


In [6]:
stoi = {ch:i for i,ch in enumerate(chars)}
itos = {i:ch for i,ch in enumerate(chars)}
encode = lambda s :[stoi[c] for c in s]
decoder = lambda l: ''.join([itos[i] for i in l])
print(encode("hii there"))
print(decoder(encode("hii there")))

[49, 50, 50, 1, 60, 49, 46, 58, 46]
hii there


In [7]:
import torch
data = torch.tensor(encode(text),dtype=torch.long)
print(data.shape,data.dtype)
print(data[:1000])

torch.Size([62037]) torch.int64
tensor([35, 61, 43, 42, 50,  1,  8, 17,  0, 24, 63,  1, 69, 64, 70, 55, 70, 55,
         1, 59, 73, 58, 53, 42, 58, 73, 55, 42,  1, 42, 52, 73, 53,  1, 46, 58,
        54, 46, 63, 46, 55, 18,  0, 36, 61, 68, 61, 54, 61, 64, 42,  4,  1, 45,
        61, 42, 54, 73, 64, 42,  1, 69, 55, 46, 54,  1, 62, 46, 58, 54, 46, 63,
        46, 55, 18,  0, 26, 70, 55, 42, 49, 60, 42, 55,  1, 59, 42, 58, 49, 56,
        75, 60, 61, 54,  4,  1, 42, 54, 42,  1, 45, 50, 53, 46, 52, 60, 46, 55,
         1, 42, 63, 73, 52, 18,  0, 38, 54, 61, 45, 61, 54, 61,  1, 58, 42, 49,
        54, 46, 60, 50, 55, 46,  1, 43, 42, 71, 53, 42, 54, 73, 75, 73, 54,  1,
        43, 46, 55,  0,  0, 35, 61, 43, 42, 50,  1,  9, 17,  0, 21, 70, 63, 70,
        52, 59, 46,  1, 45, 46,  1, 50, 59, 63, 42, 55, 73, 54,  4,  1, 52, 69,
        60, 70, 53, 70, 52, 53, 46, 58, 50, 54,  4,  0, 40, 70, 44, 46,  1, 37,
        42, 55, 58, 73,  3,  1, 45, 42, 55,  1, 61, 54, 61, 60,  1, 52, 46, 59,
        

In [8]:
n = int(0.9*len(data))
train_data = data[:n]
val_data = data[n:]

In [9]:
block_size = 8
train_data[:block_size+1]

tensor([35, 61, 43, 42, 50,  1,  8, 17,  0])

In [10]:
x = train_data[:block_size]
y = train_data[1:block_size+1]
for t in range(block_size):
  contex=x[:t+1]
  target=y[t]
  print(f"when input is {contex} the target is {target}")

when input is tensor([35]) the target is 61
when input is tensor([35, 61]) the target is 43
when input is tensor([35, 61, 43]) the target is 42
when input is tensor([35, 61, 43, 42]) the target is 50
when input is tensor([35, 61, 43, 42, 50]) the target is 1
when input is tensor([35, 61, 43, 42, 50,  1]) the target is 8
when input is tensor([35, 61, 43, 42, 50,  1,  8]) the target is 17
when input is tensor([35, 61, 43, 42, 50,  1,  8, 17]) the target is 0


In [13]:
torch.manual_seed(1337)
batch_size = 4
block_size= 8
def get_batch(split):
  data = train_data if split == 'train' else val_data
  ix = torch.randint(len(data) - block_size ,(batch_size,))
  x = torch.stack([data[i:i+block_size] for i in ix])
  y = torch.stack([data[i+1:block_size+i+1] for i in ix])
  x,y = x.to(device),y.to(device)
  return x,y

xb,yb=get_batch("train")
print("inputs")
print(xb.shape)
print(xb)
print("targets")
print(yb.shape)
print(yb)

print("-------")


for b in range(batch_size):
  for t in range(block_size):
    context = xb[b,:t+1]
    target = yb[b,t]
    print(f"when input is {context.tolist()} the target is {target}")

inputs
torch.Size([4, 8])
tensor([[55, 50, 55,  1, 48, 50, 43, 50],
        [43, 42, 50,  1,  9, 13, 17,  0],
        [73, 53,  1, 56, 53, 59, 42, 17],
        [70, 63, 46,  1, 68, 73, 52, 54]], device='cuda:0')
targets
torch.Size([4, 8])
tensor([[50, 55,  1, 48, 50, 43, 50, 63],
        [42, 50,  1,  9, 13, 17,  0, 25],
        [53,  1, 56, 53, 59, 42, 17,  0],
        [63, 46,  1, 68, 73, 52, 54, 73]], device='cuda:0')
-------
when input is [55] the target is 50
when input is [55, 50] the target is 55
when input is [55, 50, 55] the target is 1
when input is [55, 50, 55, 1] the target is 48
when input is [55, 50, 55, 1, 48] the target is 50
when input is [55, 50, 55, 1, 48, 50] the target is 43
when input is [55, 50, 55, 1, 48, 50, 43] the target is 50
when input is [55, 50, 55, 1, 48, 50, 43, 50] the target is 63
when input is [43] the target is 42
when input is [43, 42] the target is 50
when input is [43, 42, 50] the target is 1
when input is [43, 42, 50, 1] the target is 9
when inp

In [14]:
print(xb)

tensor([[55, 50, 55,  1, 48, 50, 43, 50],
        [43, 42, 50,  1,  9, 13, 17,  0],
        [73, 53,  1, 56, 53, 59, 42, 17],
        [70, 63, 46,  1, 68, 73, 52, 54]], device='cuda:0')


In [29]:
max_iters = 5000
eval_iters=200
eval_interval = 500
learning_rate = 3e-4
n_embed= 384
block_size = 256
batch_size=64
n_head= 6
n_layer=6
dropout=0.2

In [16]:
@torch.no_grad()
def estimation_loss():
  out = {}
  model.eval()
  for split in ["train","val"]:
    losses = torch.zeros(eval_iters)
    for k in range(eval_iters):
      X,Y = get_batch(split)
      logits,loss = model(X,Y)
      losses[k] = loss.item()
    out[split] = losses.mean()
  model.train()
  return out

In [41]:
import torch
import torch.nn as nn
from torch.nn import functional as F

torch.manual_seed(1337)

class BigramLanguageModel(nn.Module):
  def __init__(self,vocab_size):
    super().__init__()
    self.token_embedding_table = nn.Embedding(vocab_size,vocab_size)

  def forward(self,idx,targets=None):
    logits = self.token_embedding_table(idx)
    if targets is None:
      loss = None
    else:
      B , T ,C = logits.shape
      logits = logits.view(B*T,C)
      targets = targets.view(B*T)
      loss=F.cross_entropy(logits,targets)
    return logits,loss

  def generate(self,idx,max_new_tokens):
    for _ in range(max_new_tokens):
      logits,loss = self(idx)
      logits = logits[:,-1,:]
      probs = F.softmax(logits,dim=-1)
      idx_next= torch.multinomial(probs,num_samples=1)
      idx = torch.cat((idx,idx_next),dim=1)
    return idx

model = BigramLanguageModel(vocab_size)
m = model.to(device)
context = torch.zeros((1,1),dtype=torch.long,device=device)
generated_chars = decoder(m.generate(context,max_new_tokens=500)[0].tolist())
print(generated_chars)


mTşı579hLY0GCZÖugğUÜhHVlE.pLLD
g3fIbıNY7Yb6'ç38TMDdH02BgF0a5dEeV
eSYTPSRreScG-un,?Fvt1rÜVoTbSi!MAVıböpu2Z!Iğ6RK7c9iUkTCvzTılK3
Tl2,'EırOcıf3f37YC,Pj'R4iiTi
vZ
bdLf3TbK:tkhZfD4ivnİAj?üaoİ?YDpMSş74nRh ,9,?O!4ğIRgYC!O işugk;ÜOÖZ!Mü0eç02ü4agttPGÖ-9ZH4FİZı-Sşöuhy1pöiü'IOK hT7ğlmğTbdM8BdCnhd
miihTbij7ğMühCölVR08Gl;KegP8bBfY2üFİü3Ü?,ÜF'APÖ:NÖd'MeKo'; O 0EÖDdİi0vn1p
uKmhüFjçÜkoüFR!6 pü4424K9E!Nj3c-r.t4y1pÖFOÇörŞ.cı;aIL95aDTBöl711oAc0GEshİÜvöaÜY,Tö3
MeapNYRU'Mı:ş9;iTÖesfÇPDG-hU9Vo56öyALoşnİmPG,ck,P:Z!GÜ1


In [32]:
optimizer = torch.optim.AdamW(m.parameters(),lr=learning_rate)


In [43]:
for iter in range(max_iters):
  if iter % eval_interval == 0:
    losses = estimation_loss()
    print(f"step: {iter}: train loss: {losses['train']:.4f}, val loss: {losses['val']:.4f}")
  xb,yb=get_batch("train")
  logits,loss = m(xb,yb)
  optimizer.zero_grad(set_to_none=True)
  loss.backward()
  optimizer.step()
context = torch.zeros((1,1),dtype=torch.long,device=device)
print(decoder(m.generate(context,max_new_tokens=500)[0].tolist()))

step: 0: train loss: 4.8147, val loss: 4.7916
step: 300: train loss: 3.2636, val loss: 3.2710
step: 600: train loss: 2.7579, val loss: 2.7843
step: 900: train loss: 2.6145, val loss: 2.5851
step: 1200: train loss: 2.5172, val loss: 2.5422
step: 1500: train loss: 2.4924, val loss: 2.4832
step: 1800: train loss: 2.4595, val loss: 2.4750
step: 2100: train loss: 2.4594, val loss: 2.4900
step: 2400: train loss: 2.4246, val loss: 2.4601
step: 2700: train loss: 2.4412, val loss: 2.4457

Fön:
Neklda Cağüni ötın kedurany nlık ba eyar0:
Dışk;

Rubirımırdeyün m zdi ar bari nçirı va:
Ne soköz cemayapırsezüp imersı:
Vp üzaisi tinsöçlendamaşa bertitir köğam ndİEylilı giholtar diyluşa san sendon bLzeralekira' d88:ütehehekenndül i sanetı.
Sovidın;

Güme banur mdututt:

Guyla h bi tletsem k;
Rubanilubirünlıva tızubuşt akubadehimerı sen naş,

Şaklkak bar;yolm bubanlavefsilırklıklakt binsada vginyrıkla unıkldedalir a' di 2:

Alubihir mün ne t, ının daçirur da, nsTağlır gök 35:4nyl hendıleb


In [64]:
torch.manual_seed(1337)
B,T,C = 4,8,2
x = torch.randn(B,T,C)
x.shape

RuntimeError: CUDA error: device-side assert triggered
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.


In [46]:
xbow = torch.zeros((B,T,C))
for b in range(B):
  for t in range(T):
    xprev=x[b,:t+1]
    xbow[b,t] = torch.mean(xprev,0)

In [53]:
wei = torch.tril(torch.ones(T,T))
wei = wei / wei.sum(1,keepdim=True)
xbow2 = wei @ x
torch.allclose(xbow,xbow2)

False

In [47]:
x[0]

tensor([[ 0.1808, -0.0700],
        [-0.3596, -0.9152],
        [ 0.6258,  0.0255],
        [ 0.9545,  0.0643],
        [ 0.3612,  1.1679],
        [-1.3499, -0.5102],
        [ 0.2360, -0.2398],
        [-0.9211,  1.5433]])

In [48]:
xbow[0]

tensor([[ 0.1808, -0.0700],
        [-0.0894, -0.4926],
        [ 0.1490, -0.3199],
        [ 0.3504, -0.2238],
        [ 0.3525,  0.0545],
        [ 0.0688, -0.0396],
        [ 0.0927, -0.0682],
        [-0.0341,  0.1332]])

In [50]:
torch.tril(torch.ones(3,3))

tensor([[1., 0., 0.],
        [1., 1., 0.],
        [1., 1., 1.]])

In [52]:
torch.manual_seed(42)
a=torch.tril(torch.ones(3,3))
a= a / torch.sum(a,1,keepdim=True)
b=torch.randint(0,10,(3,2)).float()
c = a @ b
print("a=")
print(a)
print("-------")
print("b=")
print(b)
print("-------")
print("c=")
print(c)

a=
tensor([[1.0000, 0.0000, 0.0000],
        [0.5000, 0.5000, 0.0000],
        [0.3333, 0.3333, 0.3333]])
-------
b=
tensor([[2., 7.],
        [6., 4.],
        [6., 5.]])
-------
c=
tensor([[2.0000, 7.0000],
        [4.0000, 5.5000],
        [4.6667, 5.3333]])


In [55]:
tril = torch.tril(torch.ones(T,T))
wei = torch.zeros((T,T))
wei = wei.masked_fill(tril==0,float('-inf'))
wei = F.softmax(wei,dim=-1)
xbow3= wei @ x
torch.allclose(xbow2,xbow3)

True

In [19]:
import torch
import torch.nn as nn
from torch.nn import functional as F

In [28]:
class Head(nn.Module):
  def __init__(self,head_size):
    super().__init__()
    self.key = nn.Linear(n_embed,head_size,bias=False)
    self.query=nn.Linear(n_embed,head_size,bias=False)
    self.value=nn.Linear(n_embed,head_size,bias=False)
    self.register_buffer('tril',torch.tril(torch.ones(block_size,block_size)))
    self.dropout = nn.Dropout(0.2)
  def forward(self,x):
    B,T,C = x.shape
    k = self.key(x)
    q = self.query(x)
    wei = q @ k.transpose(-2,-1) * C**-0.5
    wei = wei.masked_fill(self.tril[:T,:T]==0,float('-inf'))
    wei = F.softmax(wei,dim=-1)
    wei = self.dropout(wei)
    v = self.value(x)
    out = wei @ v
    return out

In [26]:
class MultiHeadAttention(nn.Module):
  def __init__(self,num_heads,head_size):
    super().__init__()
    self.heads = nn.ModuleList([Head(head_size) for _ in range(num_heads)])
    self.proj = nn.Linear(n_embed,n_embed)
    self.dropout = nn.Dropout(0.2)

  def forward(self,x):
    out = torch.cat([h(x) for h in self.heads],dim=-1)
    out = self.dropout(self.proj(out))
    return out


In [27]:
class FeedForward(nn.Module):
  def __init__(self,n_embed):
    super().__init__()
    self.net = nn.Sequential(
        nn.Linear(n_embed,4*n_embed),
        nn.ReLU(),
        nn.Linear(4*n_embed,n_embed),
        nn.Dropout(0.2)
    )
  def forward(self,x):
    return self.net(x)

In [25]:
class Blocks(nn.Module):
  def __init__(self,n_embed,n_head):
    super().__init__()
    head_size = n_embed // n_head
    self.sa = MultiHeadAttention(n_head,head_size)
    self.ffwd = FeedForward(n_embed)
    self.ln1 = nn.LayerNorm(n_embed)
    self.ln2 = nn.LayerNorm(n_embed)

  def forward(self,x):
    x = x + self.sa(self.ln1(x))
    x = x + self.ffwd(self.ln2(x))
    return x

In [30]:
import torch
import torch.nn as nn
from torch.nn import functional as F

class Head(nn.Module):
  def __init__(self,head_size):
    super().__init__()
    self.key = nn.Linear(n_embed,head_size,bias=False)
    self.query=nn.Linear(n_embed,head_size,bias=False)
    self.value=nn.Linear(n_embed,head_size,bias=False)
    self.register_buffer('tril',torch.tril(torch.ones(block_size,block_size)))
  def forward(self,x):
    B,T,C = x.shape
    k = self.key(x)
    q = self.query(x)
    wei = q @ k.transpose(-2,-1) * C**-0.5
    wei = wei.masked_fill(self.tril[:T,:T]==0,float('-inf'))
    wei = F.softmax(wei,dim=-1)
    v = self.value(x)
    out = wei @ v
    return out

class BigramLanguageModel(nn.Module):
  def __init__(self):
    super().__init__()
    self.token_embedding_table = nn.Embedding(vocab_size,n_embed)
    self.position_embedding_table = nn.Embedding(block_size,n_embed)
    self.blocks = nn.Sequential(*[Blocks(n_embed,n_head=n_head) for _ in range(n_layer)])
    self.ln_f = nn.LayerNorm(n_embed)
    self.lm_head = nn.Linear(n_embed,vocab_size)

  def forward(self,idx,targets=None):
    B,T = idx.shape
    token_emb = self.token_embedding_table(idx)
    pos_emb = self.position_embedding_table(torch.arange(T,device=device))
    x = token_emb + pos_emb
    x=self.blocks(x)
    x=self.ln_f(x)
    logits = self.lm_head(x)
    if targets is None:
      loss = None
    else:
      B , T ,C = logits.shape
      logits = logits.view(B*T,C)
      targets = targets.view(B*T)
      loss=F.cross_entropy(logits,targets)
    return logits,loss

  def generate(self,idx,max_new_tokens):
    for _ in range(max_new_tokens):
      idx_cond = idx[:,-block_size:]
      logits,loss = self(idx_cond)
      logits = logits[:,-1,:]
      probs = F.softmax(logits,dim=-1)
      idx_next= torch.multinomial(probs,num_samples=1)
      idx = torch.cat((idx,idx_next),dim=1)
    return idx

model = BigramLanguageModel()
m = model.to(device)
context = torch.zeros((1,1),dtype=torch.long,device=device)
generated_chars = decoder(m.generate(context,max_new_tokens=500)[0].tolist())
print(generated_chars)


a.FHss nV27Gır4Ö;ü4H,3hIMGgv6ÖO50ÇMPAP;ö 0şHHjM:SşöNfvyIsnş-8ÇŞzYaBr2ğIiIü'4FcugNTv3bİAnYZDaHçşvŞ;İGRNğ18tÖHiYkNdFÖpıt
3H7:U,1u?T.dÖ0dNDı;cZf8TS7.Vıüüt4k4CMCğ:rOÜHI,ŞP0,TsdN!Bd58sktjad
OpP?8ju5,ğ3Bmt!tfZÖıaİ.P:h6fE-cOGe
Yö:T2GgşÖR-CUP:F!öÖA28ffItDPzİsd0kd2Hm
ÇI!Fc
SY0z0cKaİB,Y0Ik8öjşyÖ?Ö9ğggm!pÇeğ2Ü3cLmln0KBNğ:p
IİiövfKKİNicndYİşHödm3KC87pY'81VşB95'!phrAB'.öY;Gıye
ö0:cöRv5EÇnüMp34YeÖAçuSY0pİü'ö3G1çIb;öa
ğ4Bc:bBBeYy5ğ3E'1çF0:jÇu1B5cA04u;PfdÖ7Ü'3-ğöÖua;ömpEZ8jBlhy5-I,ı NcuOgNYğ??
çI-!cdö:LN3föcj!ı


In [33]:
for iter in range(max_iters):
  if iter % eval_interval == 0:
    losses = estimation_loss()
    print(f"step: {iter}: train loss: {losses['train']:.4f}, val loss: {losses['val']:.4f}")
  xb,yb=get_batch("train")
  logits,loss = m(xb,yb)
  optimizer.zero_grad(set_to_none=True)
  loss.backward()
  optimizer.step()
context = torch.zeros((1,1),dtype=torch.long,device=device)
print(decoder(m.generate(context,max_new_tokens=500)[0].tolist()))

step: 0: train loss: 4.5017, val loss: 4.5016
step: 500: train loss: 1.7423, val loss: 1.8774
step: 1000: train loss: 0.9659, val loss: 2.0056
step: 1500: train loss: 0.2119, val loss: 3.1427
step: 2000: train loss: 0.0937, val loss: 3.8597
step: 2500: train loss: 0.0730, val loss: 4.2735
step: 3000: train loss: 0.0637, val loss: 4.5403
step: 3500: train loss: 0.0571, val loss: 4.7067
step: 4000: train loss: 0.0545, val loss: 4.8597
step: 4500: train loss: 0.0521, val loss: 4.9284


Rubai 308:
Her gün şarap cümbüşüne dalanların da
Her gece mihrap önünde kalanların da
Islanmaya yok, yok.
Key ve de ba, çünkü yükse ver diler önünde.

Rubai 132:
Bu dünya gönül dedenin canımın kanlar
Ne hamutlu kanlar halk varamış bir zaman
Senda bu şerde görer gibi ocak önünden.

Rubai 120:
Tanrı' ı55:
Yokların dolan mezaneyin;
Ama Maklanları önümün derinin;
Bunların mi virgi kader cim deli, sana gibi;
Bu dünyadık yerde kalması yok, yağmur.

Rubai 154:
Kimi de imiş ki bir taprsın kana:
Bir git


In [34]:
m.save_pretrained("model")

AttributeError: 'BigramLanguageModel' object has no attribute 'save_pretrained'